# 해당 파일 설명

이 코드를 실행하기 위한 파일, /convert 경로에 다음의 파일이 필요합니다.

1. plant_test_all_weather_feature_add_cluster_add_out_in.csv
2. date_trend_point.csv

이 코드를 실행하기 위한 파일, /model 경로에 다음의 파일이 필요합니다.

1. model_cond_all_es1000.pkl

이 코드를 모두 실행하고 나면, /result 경로에 다음의 파일이 생성됩니다.

1. es1000_th_08.csv

## 추가 설명

test 데이터를 model_cond_all_es1000 을 통해 cond_loc를 예측하여

그 결과를 제출형식과 맞게 만들어 result폴더로 저장합니다.

# 1. Google drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
# BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)


Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted User Data List =====\n" ,check_output(["ls", DATA_CONV_USER_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))
print("===== Model List =====\n" ,check_output(["ls", MODEL_PATH]).decode("utf8"))
print("===== Result file List =====\n" ,check_output(["ls", RESULT_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted User Data List =====
 backup_0.96model
data_all.csv
data_all_out.csv
data_all_out_in.csv
date_trend.csv
date_trend_point.csv
ExtraTreesClassifier_th_10.csv
GradientBoostingClassifier_all_feature_th_40.csv
lastplus_th_10.csv
last_th_08.csv
last_th_10.csv
model
model_cond_all.pkl
model_cond_all_plus_.pkl
model_hum_in.pkl
model_hum_out_loc.pkl
model_tem_coil.pkl
model_tem_in.pkl
model_tem_out_loc.pkl
plant1_by_loc.csv
plant1_by_loc.gsheet
plant2_by_loc.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster_add_out_in.gsheet
plant_train_all_weather_feature_add_cluster.csv
predict.csv
RandomForestClassifier_th_10.csv
result.c

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import copy


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
import lightgbm

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

# interpolation 함수
import scipy.interpolate as spi

# model load
import pickle
from sklearn.externals import joblib

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Get the Data

## 1. plant data 가져오기

In [ ]:
# plant 데이터 
#data_train = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_train_all_weather_feature_add_cluster.csv"))
data_test = pd.read_csv(os.path.join(DATA_CONV_PATH, "plant_test_all_weather_feature_add_cluster_add_out_in.csv"))

# unnamed: 0 제거
#data_train.drop(['Unnamed: 0'], axis='columns', inplace=True)
data_test.drop(['Unnamed: 0'], axis='columns', inplace=True)

# date 변수 str -> datetime.date()
#data_train['date'] = data_train['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date()) 
data_test['date'] = data_test['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date()) 

data_test

,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,seosan-ta-changed,seosan-hm-changed,seosan-ground-changed,tem_out_loc,hum_out_loc,tem_in,hum_in,tem_coil
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,4.1289,53.8826,8.5984,38.3108,9.2800
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,1.5862,60.7794,5.7104,46.2029,6.6250
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,14.6972,28.9453,10.8321,40.6705,10.1216
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,12.4695,29.7087,11.4784,36.5843,9.7326
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,12.1573,28.7545,10.7778,35.4266,7.9725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,4.3119,61.0764,8.9442,49.5774,9.7412
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,2.5650,64.9708,8.5004,44.7795,9.9303
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,3.1238,65.0836,7.2061,48.8123,8.6168
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,12.8558,34.6704,13.9568,39.8056,11.8661


In [ ]:
# date 별로 in, out 클러스터링 데이터
data_cluster = pd.read_csv(os.path.join(DATA_CONV_PATH, "date_trend_point.csv"))

data_cluster.drop(['Unnamed: 0'], axis='columns', inplace=True)

data_cluster['date'] = data_cluster['date'].apply(lambda x : datetime.strptime(x + " 00:00:00", '%Y-%m-%d %H:%M:%S').date()) 

data_cluster

,date,plant,loc,tem_out_loc-trend,hum_out_loc-trend,tem_in-trend,hum_in-trend,tem_coil-trend,tem_out_loc-point,hum_out_loc-point,tem_in-point,hum_in-point,tem_coil-point
0,2016-04-01,1,1,2,4,2,0,2,1,1,2,4,1
1,2016-04-01,1,2,4,2,3,3,0,3,3,3,0,1
2,2016-04-01,1,3,2,1,3,3,2,3,1,2,2,3
3,2016-04-01,2,1,3,3,3,4,3,3,2,2,0,2
4,2016-04-01,2,2,1,4,0,1,3,1,2,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8761,2020-03-31,1,2,4,2,0,0,0,4,3,2,0,1
8762,2020-03-31,1,3,2,1,0,0,0,2,1,1,2,3
8763,2020-03-31,2,1,3,3,0,0,3,2,2,0,0,1
8764,2020-03-31,2,2,1,4,3,0,3,0,2,1,4,4


In [ ]:
data_test = pd.merge(data_test, data_cluster, how='left', on = ['date', 'plant', 'loc'])
data_test

,mea_ddhr,plant,loc,date,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,seosan-ta-changed,seosan-hm-changed,seosan-ground-changed,tem_out_loc,hum_out_loc,tem_in,hum_in,tem_coil,tem_out_loc-trend,hum_out_loc-trend,tem_in-trend,hum_in-trend,tem_coil-trend,tem_out_loc-point,hum_out_loc-point,tem_in-point,hum_in-point,tem_coil-point
0,2019-04-02 00:00:00,2,3,2019-04-02,0.0,1.70,53.90,0.60,0.0,3.00,1.50,0.0,1020.90,1023.40,-0.60,64.0,0.80,0.0,-6.50,3.80,1020.50,1023.80,0.0,0.000,0.50,8.90,37.0,4.00,0.0,8.90,37.0,4.00,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,4.1289,53.8826,8.5984,38.3108,9.2800,2,2,0,2,3,1,2,1,2,1
1,2019-04-02 04:30:00,1,2,2019-04-02,270.0,0.40,66.85,1.05,0.0,-2.15,1.35,0.0,1021.05,1023.55,-2.45,81.0,0.65,0.0,-5.20,4.15,1020.45,1023.75,0.0,0.000,-0.65,11.15,25.0,3.20,0.0,11.15,25.0,3.20,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,1.5862,60.7794,5.7104,46.2029,6.6250,4,2,0,4,4,4,3,2,0,1
2,2019-04-02 10:30:00,2,3,2019-04-02,630.0,9.55,39.00,2.75,0.0,9.95,2.95,0.0,1022.05,1024.45,9.85,38.0,3.70,0.0,-4.30,4.60,1021.60,1024.80,1.0,2.445,18.90,5.05,46.0,2.30,0.0,5.05,46.0,2.30,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,14.6972,28.9453,10.8321,40.6705,10.1216,2,2,0,2,3,1,2,1,2,1
3,2019-04-02 11:00:00,1,1,2019-04-02,660.0,10.20,28.20,3.20,0.0,10.70,3.00,0.0,1021.90,1024.30,10.40,27.0,4.40,0.0,-7.80,3.40,1021.50,1024.70,1.0,2.750,21.00,4.90,50.0,2.20,0.0,4.90,50.0,2.20,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,12.4695,29.7087,11.4784,36.5843,9.7326,2,4,0,3,0,3,1,0,4,1
4,2019-04-02 13:30:00,2,1,2019-04-02,810.0,10.15,24.50,4.60,0.0,11.15,4.45,0.0,1020.75,1023.15,10.90,23.0,5.50,0.0,-9.40,3.00,1020.60,1023.80,1.0,3.280,24.25,3.40,59.0,2.30,0.0,3.40,59.0,2.30,0.0,1,4,3,4,1,1,14.797798,66.997998,25.195996,12.1573,28.7545,10.7778,35.4266,7.9725,3,3,0,0,2,2,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,2020-03-31,180.0,2.90,65.80,1.00,0.0,2.30,1.40,0.0,1018.60,1021.10,0.80,83.0,0.00,0.0,-1.70,5.40,1018.20,1021.40,0.0,0.000,2.70,15.70,42.0,3.60,0.0,15.70,42.0,3.60,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,4.3119,61.0764,8.9442,49.5774,9.7412,4,2,0,0,0,4,3,2,0,1
6749,2020-03-31 05:00:00,2,3,2020-03-31,300.0,0.50,72.40,0.30,0.0,0.30,0.60,0.0,1017.50,1020.00,0.20,84.0,1.60,0.0,-2.10,5.20,1016.30,1019.50,0.0,0.000,2.00,14.90,33.0,2.90,0.0,14.90,33.0,2.90,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,2.5650,64.9708,8.5004,44.7795,9.9303,2,2,0,2,3,1,2,1,2,1
6750,2020-03-31 07:00:00,2,1,2020-03-31,420.0,0.60,74.80,0.10,0.0,-0.10,1.20,0.0,1017.60,1020.10,0.40,82.0,0.30,0.0,-2.30,5.20,1016.90,1020.10,0.1,0.070,1.90,12.30,33.0,3.10,0.0,12.30,33.0,3.10,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,3.1238,65.0836,7.2061,48.8123,8.6168,3,3,0,0,3,2,2,0,0,1
6751,2020-03-31 11:30:00,1,1,2020-03-31,690.0,14.50,32.30,1.60,0.0,13.60,1.40,0.0,1017.25,1019.60,15.20,25.0,1.55,0.0,-4.90,4.30,1016.35,1019.35,1.0,2.880,28.05,7.35,68.5,1.10,0.0,7.35,68.5,1.10,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,12.8558,34.6704,13.9568,39.8056,11.8661,2,4,0,2,4,3,1,0,4,1


# Modeling


In [ ]:
# 예측에 필요한 변수 정의
data_columns = list(data_test.columns)
remove_list = ['mea_ddhr', 'date']
for rmv in remove_list:
  data_columns.remove(rmv)
data_columns

['plant',
 'loc',
 'time',
 'dangjin-ta',
 'dangjin-hm',
 'dangjin-ws',
 'dangjin-rn',
 'sinpyeong-ta',
 'sinpyeong-ws',
 'sinpyeong-rn',
 'sinpyeong-lp',
 'sinpyeong-ap',
 'seosan-ta',
 'seosan-hm',
 'seosan-ws',
 'seosan-rn',
 'seosan-dew',
 'seosan-sp',
 'seosan-lp',
 'seosan-ap',
 'seosan-sr',
 'seosan-ss',
 'seosan-ground',
 'songsan-ta',
 'songsan-hm',
 'songsan-ws',
 'songsan-rn',
 'songak-ta',
 'songak-hm',
 'songak-ws',
 'songak-rn',
 'seosan-ta-trend',
 'seosan-hm-trend',
 'seosan-ground-trend',
 'seosan-ta-point',
 'seosan-hm-point',
 'seosan-ground-point',
 'seosan-ta-changed',
 'seosan-hm-changed',
 'seosan-ground-changed',
 'tem_out_loc',
 'hum_out_loc',
 'tem_in',
 'hum_in',
 'tem_coil',
 'tem_out_loc-trend',
 'hum_out_loc-trend',
 'tem_in-trend',
 'hum_in-trend',
 'tem_coil-trend',
 'tem_out_loc-point',
 'hum_out_loc-point',
 'tem_in-point',
 'hum_in-point',
 'tem_coil-point']

In [ ]:
# 예측에 필요한 변수 정의 --> 순서 까지 포함
data_columns = ['plant',
 'loc',
 'tem_in',
 'hum_in',
 'tem_coil',
 'tem_out_loc',
 'hum_out_loc',
 'time',
 'dangjin-ta',
 'dangjin-hm',
 'dangjin-ws',
 'dangjin-rn',
 'sinpyeong-ta',
 'sinpyeong-ws',
 'sinpyeong-rn',
 'sinpyeong-lp',
 'sinpyeong-ap',
 'seosan-ta',
 'seosan-hm',
 'seosan-ws',
 'seosan-rn',
 'seosan-dew',
 'seosan-sp',
 'seosan-lp',
 'seosan-ap',
 'seosan-sr',
 'seosan-ss',
 'seosan-ground',
 'songsan-ta',
 'songsan-hm',
 'songsan-ws',
 'songsan-rn',
 'songak-ta',
 'songak-hm',
 'songak-ws',
 'songak-rn',
 'seosan-ta-trend',
 'seosan-hm-trend',
 'seosan-ground-trend',
 'seosan-ta-point',
 'seosan-hm-point',
 'seosan-ground-point',
 'seosan-ta-changed',
 'seosan-hm-changed',
 'seosan-ground-changed',
 'tem_out_loc-trend',
 'hum_out_loc-trend',
 'tem_in-trend',
 'hum_in-trend',
 'tem_coil-trend',
 'tem_out_loc-point',
 'hum_out_loc-point',
 'tem_in-point',
 'hum_in-point',
 'tem_coil-point']
data_columns

['plant',
 'loc',
 'tem_in',
 'hum_in',
 'tem_coil',
 'tem_out_loc',
 'hum_out_loc',
 'time',
 'dangjin-ta',
 'dangjin-hm',
 'dangjin-ws',
 'dangjin-rn',
 'sinpyeong-ta',
 'sinpyeong-ws',
 'sinpyeong-rn',
 'sinpyeong-lp',
 'sinpyeong-ap',
 'seosan-ta',
 'seosan-hm',
 'seosan-ws',
 'seosan-rn',
 'seosan-dew',
 'seosan-sp',
 'seosan-lp',
 'seosan-ap',
 'seosan-sr',
 'seosan-ss',
 'seosan-ground',
 'songsan-ta',
 'songsan-hm',
 'songsan-ws',
 'songsan-rn',
 'songak-ta',
 'songak-hm',
 'songak-ws',
 'songak-rn',
 'seosan-ta-trend',
 'seosan-hm-trend',
 'seosan-ground-trend',
 'seosan-ta-point',
 'seosan-hm-point',
 'seosan-ground-point',
 'seosan-ta-changed',
 'seosan-hm-changed',
 'seosan-ground-changed',
 'tem_out_loc-trend',
 'hum_out_loc-trend',
 'tem_in-trend',
 'hum_in-trend',
 'tem_coil-trend',
 'tem_out_loc-point',
 'hum_out_loc-point',
 'tem_in-point',
 'hum_in-point',
 'tem_coil-point']

In [ ]:
# cond_loc 예측 모델 가지고 오기
model_cond = joblib.load(os.path.join(MODEL_PATH, "model_cond_all_es1000.pkl"))
model_cond.keys()

dict_keys([('ExtraTreesClassifier', 'cond_loc')])

In [ ]:
# 예측하기
data_test_cond = data_test[['mea_ddhr', 'plant', 'loc']]

for key in model_cond.keys():
  model = model_cond[key]
  data_test_cond[key[0]] = model.predict_proba(data_test[data_columns])[:, 1]
data_test_cond

,mea_ddhr,plant,loc,ExtraTreesClassifier
0,2019-04-02 00:00:00,2,3,0.000
1,2019-04-02 04:30:00,1,2,0.000
2,2019-04-02 10:30:00,2,3,0.000
3,2019-04-02 11:00:00,1,1,0.000
4,2019-04-02 13:30:00,2,1,0.002
...,...,...,...,...
6748,2020-03-31 03:00:00,1,2,0.000
6749,2020-03-31 05:00:00,2,3,0.000
6750,2020-03-31 07:00:00,2,1,0.000
6751,2020-03-31 11:30:00,1,1,0.001


In [ ]:
data_test_cond.columns = ['mea_ddhr', 'plant', 'loc', 'cond_loc']

In [ ]:
def makeTestSubmitDateFormatEqual(df):
  """
  describe : test 의 mea_ddhr을 submit의 mea_ddhr의 양식과 같게 만드는 함수

  <input>
  df = pd.dataframe -> test 데이터

  <output>
  df = pd.dataframe -> test 데이터의 mea_ddhr이 변환된 df
  """
  ret_df = copy.deepcopy(df)
  for i in range(ret_df.shape[0]):
    ddhr = ret_df.loc[i, "mea_ddhr"]
    date = ddhr.split()[0]
    times = (ddhr.split()[1]).split(":")

    hour = str(int(times[0]))
    minute = times[1]
    new_time = hour + ":" + minute
    ret_df.loc[i, "mea_ddhr"] = date + " " + new_time
  return ret_df

In [ ]:
data_predicted = makeTestSubmitDateFormatEqual(data_test_cond)
data_predicted

,mea_ddhr,plant,loc,cond_loc
0,2019-04-02 0:00,2,3,0.000
1,2019-04-02 4:30,1,2,0.000
2,2019-04-02 10:30,2,3,0.000
3,2019-04-02 11:00,1,1,0.000
4,2019-04-02 13:30,2,1,0.002
...,...,...,...,...
6748,2020-03-31 3:00,1,2,0.000
6749,2020-03-31 5:00,2,3,0.000
6750,2020-03-31 7:00,2,1,0.000
6751,2020-03-31 11:30,1,1,0.001


In [ ]:
# submit load
data_submit = pd.read_csv(os.path.join(DATA_RAW_PATH, "submit.csv"))
data_submit.drop(['X24H_COND_LOC','X24H_COND_LOC_PROB','X48H_COND_LOC','X48H_COND_LOC_PROB'], axis='columns', inplace=True)
# data_submit = stringToDatetime(data_submit, 'X24H_TMA')
# data_submit = stringToDatetime(data_submit, 'X48H_TMA')
data_submit['X24H_TMA'] = data_submit['X24H_TMA'].apply(lambda x : str(x))
data_submit['X48H_TMA'] = data_submit['X48H_TMA'].apply(lambda x : str(x))
data_submit

,MEA_DDHR,PLANT,LOC,X24H_TMA,X48H_TMA
0,2019-04-01 0:00,2,3,2019-04-02 0:00,2019-04-03 0:00
1,2019-04-01 4:30,1,2,2019-04-02 4:30,2019-04-03 4:30
2,2019-04-01 10:30,2,3,2019-04-02 10:30,2019-04-03 10:30
3,2019-04-01 11:00,1,1,2019-04-02 11:00,2019-04-03 11:00
4,2019-04-01 13:30,2,1,2019-04-02 13:30,2019-04-03 13:30
...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,2020-03-31 3:00
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,2020-03-31 5:00
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,2020-03-31 7:00
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,2020-03-31 11:30


In [ ]:
data_predicted.columns = ['X24H_TMA','PLANT','LOC','X24H_COND_LOC_PROB']
data_predicted

,X24H_TMA,PLANT,LOC,X24H_COND_LOC_PROB
0,2019-04-02 0:00,2,3,0.000
1,2019-04-02 4:30,1,2,0.000
2,2019-04-02 10:30,2,3,0.000
3,2019-04-02 11:00,1,1,0.000
4,2019-04-02 13:30,2,1,0.002
...,...,...,...,...
6748,2020-03-31 3:00,1,2,0.000
6749,2020-03-31 5:00,2,3,0.000
6750,2020-03-31 7:00,2,1,0.000
6751,2020-03-31 11:30,1,1,0.001


In [ ]:
data_submit = pd.merge(data_submit, data_predicted, how = 'left', on=['X24H_TMA', 'PLANT', 'LOC'])
data_submit

,MEA_DDHR,PLANT,LOC,X24H_TMA,X48H_TMA,X24H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,2019-04-03 0:00,0.000
1,2019-04-01 4:30,1,2,2019-04-02 4:30,2019-04-03 4:30,0.000
2,2019-04-01 10:30,2,3,2019-04-02 10:30,2019-04-03 10:30,0.000
3,2019-04-01 11:00,1,1,2019-04-02 11:00,2019-04-03 11:00,0.000
4,2019-04-01 13:30,2,1,2019-04-02 13:30,2019-04-03 13:30,0.002
...,...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,2020-03-31 3:00,0.000
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,2020-03-31 5:00,0.001
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,2020-03-31 7:00,0.000
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,2020-03-31 11:30,0.000


In [ ]:
data_predicted.columns = ['X48H_TMA','PLANT','LOC','X48H_COND_LOC_PROB']
data_predicted

,X48H_TMA,PLANT,LOC,X48H_COND_LOC_PROB
0,2019-04-02 0:00,2,3,0.000
1,2019-04-02 4:30,1,2,0.000
2,2019-04-02 10:30,2,3,0.000
3,2019-04-02 11:00,1,1,0.000
4,2019-04-02 13:30,2,1,0.002
...,...,...,...,...
6748,2020-03-31 3:00,1,2,0.000
6749,2020-03-31 5:00,2,3,0.000
6750,2020-03-31 7:00,2,1,0.000
6751,2020-03-31 11:30,1,1,0.001


In [ ]:
data_submit = pd.merge(data_submit, data_predicted, how = 'left', on=['X48H_TMA', 'PLANT', 'LOC'])
data_submit

,MEA_DDHR,PLANT,LOC,X24H_TMA,X48H_TMA,X24H_COND_LOC_PROB,X48H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,2019-04-03 0:00,0.000,0.000
1,2019-04-01 4:30,1,2,2019-04-02 4:30,2019-04-03 4:30,0.000,0.000
2,2019-04-01 10:30,2,3,2019-04-02 10:30,2019-04-03 10:30,0.000,0.002
3,2019-04-01 11:00,1,1,2019-04-02 11:00,2019-04-03 11:00,0.000,0.001
4,2019-04-01 13:30,2,1,2019-04-02 13:30,2019-04-03 13:30,0.002,0.000
...,...,...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,2020-03-31 3:00,0.000,0.000
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,2020-03-31 5:00,0.001,0.000
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,2020-03-31 7:00,0.000,0.000
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,2020-03-31 11:30,0.000,0.001


In [ ]:
threshold = 0.08

In [ ]:
data_submit['X24H_COND_LOC'] = (data_submit['X24H_COND_LOC_PROB'] >= threshold).apply(lambda x : 1 if x == True else 0)
print("24시간 이후 예측한 true의 개수는 :", sum(data_submit['X24H_COND_LOC'] == 1) , "개 입니다.")

24시간 이후 예측한 true의 개수는 : 256 개 입니다.


In [ ]:
data_submit['X48H_COND_LOC'] = (data_submit['X48H_COND_LOC_PROB'] >= threshold).apply(lambda x : 1 if x == True else 0)
print("48시간 이후 예측한 true의 개수는 :", sum(data_submit['X48H_COND_LOC'] == 1) , "개 입니다.")

48시간 이후 예측한 true의 개수는 : 231 개 입니다.


In [ ]:
for threshold in np.arange(0.06, 0.1, 0.001):
  threshold = round(threshold,4)
  data_submit['X24H_COND_LOC'] = (data_submit['X24H_COND_LOC_PROB'] >= threshold).apply(lambda x : 1 if x == True else 0)
  data_submit['X48H_COND_LOC'] = (data_submit['X48H_COND_LOC_PROB'] >= threshold).apply(lambda x : 1 if x == True else 0)
  print(threshold, sum(data_submit['X24H_COND_LOC'] == 1), sum(data_submit['X48H_COND_LOC'] == 1))

0.06 310 268
0.061 302 265
0.062 297 262
0.063 292 259
0.064 289 258
0.065 286 257
0.066 284 255
0.067 282 250
0.068 280 248
0.069 275 245
0.07 273 245
0.071 272 245
0.072 271 245
0.073 267 243
0.074 264 242
0.075 264 242
0.076 263 240
0.077 258 236
0.078 257 234
0.079 257 233
0.08 256 231
0.081 254 230
0.082 253 227
0.083 249 226
0.084 247 223
0.085 245 220
0.086 241 216
0.087 240 215
0.088 235 211
0.089 233 209
0.09 228 206
0.091 227 203
0.092 224 200
0.093 221 199
0.094 220 198
0.095 219 196
0.096 218 196
0.097 215 194
0.098 211 193
0.099 210 192
0.1 210 191


In [ ]:
# 제출 양식에 맞게 column을 재정렬한다.
data_submit = data_submit[["MEA_DDHR", "PLANT",	"LOC", "X24H_TMA", "X24H_COND_LOC",	"X24H_COND_LOC_PROB",	"X48H_TMA", "X48H_COND_LOC",	"X48H_COND_LOC_PROB"]]
data_submit

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,0,0.000,2019-04-03 0:00,0,0.000
1,2019-04-01 4:30,1,2,2019-04-02 4:30,0,0.000,2019-04-03 4:30,0,0.000
2,2019-04-01 10:30,2,3,2019-04-02 10:30,0,0.000,2019-04-03 10:30,0,0.002
3,2019-04-01 11:00,1,1,2019-04-02 11:00,0,0.000,2019-04-03 11:00,0,0.001
4,2019-04-01 13:30,2,1,2019-04-02 13:30,0,0.002,2019-04-03 13:30,0,0.000
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,0,0.000,2020-03-31 3:00,0,0.000
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,0,0.001,2020-03-31 5:00,0,0.000
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,0,0.000,2020-03-31 7:00,0,0.000
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,0,0.000,2020-03-31 11:30,0,0.001


In [ ]:
data_submit['X24H_COND_LOC_PROB'] = data_submit['X24H_COND_LOC_PROB']*100
data_submit['X48H_COND_LOC_PROB'] = data_submit['X48H_COND_LOC_PROB']*100

In [ ]:
data_submit

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 0:00,2,3,2019-04-02 0:00,0,0.0,2019-04-03 0:00,0,0.0
1,2019-04-01 4:30,1,2,2019-04-02 4:30,0,0.0,2019-04-03 4:30,0,0.0
2,2019-04-01 10:30,2,3,2019-04-02 10:30,0,0.0,2019-04-03 10:30,0,0.2
3,2019-04-01 11:00,1,1,2019-04-02 11:00,0,0.0,2019-04-03 11:00,0,0.1
4,2019-04-01 13:30,2,1,2019-04-02 13:30,0,0.2,2019-04-03 13:30,0,0.0
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 3:00,1,2,2020-03-30 3:00,0,0.0,2020-03-31 3:00,0,0.0
3535,2020-03-29 5:00,2,3,2020-03-30 5:00,0,0.1,2020-03-31 5:00,0,0.0
3536,2020-03-29 7:00,2,1,2020-03-30 7:00,0,0.0,2020-03-31 7:00,0,0.0
3537,2020-03-29 11:30,1,1,2020-03-30 11:30,0,0.0,2020-03-31 11:30,0,0.1


In [ ]:
data_submit.to_csv(os.path.join(RESULT_PATH, 'es1000_th_08.csv'))